# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [17]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import nltk
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
import pickle

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('LabeledMessages', con=engine)
X = df.message
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # normalizing
    norm_text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenizing
    tokens = word_tokenize(norm_text)
    # lemmatizing
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tokens.append(clean_tok)
    return clean_tokens    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state=0)

In [43]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [44]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def clf_report(y_pred, y_test):
    for i in range(len(y_test.columns)):
        print ('Report for column: '+y_test.columns[i])
        print(classification_report(y_test[y_test.columns[i]], y_pred[:, i]))

In [46]:
clf_report(y_pred, y_test)

Report for column: related
             precision    recall  f1-score   support

          0       0.60      0.35      0.44      1971
          1       0.82      0.93      0.87      6627
          2       0.68      0.35      0.46        54

avg / total       0.77      0.79      0.77      8652

Report for column: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      7136
          1       0.83      0.34      0.48      1516

avg / total       0.87      0.87      0.85      8652

Report for column: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8624
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      1.00      8652

Report for column: aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      5017
          1       0.76      0.53      0.62      3635

avg / total       0.74      0.7

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
print (pipeline.get_params())

{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7fdcb0020620>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
 

In [25]:
parameters =  {'clf__estimator__min_samples_leaf': [1, 2, 4],
    'clf__estimator__min_samples_split': [2, 5, 10]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_leaf': [1, 2, 4], 'clf__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
y_pred = cv.predict(X_test)

In [34]:
clf_report(y_pred, y_test)

Report for column: related
             precision    recall  f1-score   support

          0       0.61      0.33      0.43      1971
          1       0.82      0.94      0.88      6627
          2       0.61      0.35      0.45        54

avg / total       0.77      0.79      0.77      8652

Report for column: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      7136
          1       0.83      0.37      0.51      1516

avg / total       0.87      0.88      0.86      8652

Report for column: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8624
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      1.00      8652

Report for column: aid_related
             precision    recall  f1-score   support

          0       0.73      0.87      0.79      5017
          1       0.76      0.54      0.63      3635

avg / total       0.74      0.7

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                    ])

In [10]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
clf_report(y_pred, y_test)

Report for column: related
             precision    recall  f1-score   support

          0       0.53      0.18      0.27      1971
          1       0.79      0.95      0.86      6627
          2       0.24      0.09      0.13        54

avg / total       0.73      0.77      0.72      8652

Report for column: request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      7136
          1       0.75      0.55      0.64      1516

avg / total       0.88      0.89      0.88      8652

Report for column: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8624
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      8652

Report for column: aid_related
             precision    recall  f1-score   support

          0       0.75      0.87      0.80      5017
          1       0.77      0.60      0.68      3635

avg / total       0.76      0.7

### 9. Export your model as a pickle file

In [18]:
with open('model.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.